In [1]:
from selenium import webdriver
from selenium.webdriver import ActionChains, ChromeOptions
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common import keys
from bs4 import BeautifulSoup
import time
import pandas as pd

In [2]:
def safe_click(xpath, driver, max_retry = 3):
    try:
        to_click = driver.find_element_by_xpath(xpath)
        to_click.click()
    except:
        print("ERRO NO CLICK")
        if max_retry != 0:
            time.sleep(10)
            safe_click(xpath, driver, max_retry-1)
        else:
            print('NUMERO DE TENTATIVAS EXCEDIDA')
            get_current_data()
            raise

def safe_action(xpath, action = 'right_click', max_retry = 3):
    if action == 'right_click':
        try:
            to_act = driver.find_element_by_xpath(xpath)
            actions = ActionChains(driver)
            actions.move_to_element(to_act)
            actions.context_click(to_act)
            actions.perform()
        except:
            if max_retry != 0:
                time.sleep(20)
                safe_click(xpath,  driver,max_retry-1)
            else:
                print('NUMERO DE TENTATIVAS EXCEDIDA')
                get_current_data()
                driver.quit()

In [4]:
lista_mes=['Janeiro','Fevereiro','Marco','Abril','Maio','Junho','Julho','Agosto','Setembro','Outubro','Novembro','Dezembro']

ratosus = pd.DataFrame()

for mes in lista_mes:
    options = ChromeOptions()
    options.add_argument('headless')
    driver = webdriver.Chrome('C:\Program Files (x86)\Google\Chrome\Application\chromedriver', options=options)
    driver.get('http://tabnet.datasus.gov.br/cgi/tabcgi.exe?sinannet/cnv/tubercbr.def')
    for ano in list(range(2001, 2019)):
        safe_click(f'//div[@class="periodo"]//option[contains(.,"{ano}")]', driver)
        time.sleep(1)
    safe_click('//option[@value="UF_de_notificação"]', driver)
    safe_click('(//option[@value="Ano_Diagnóstico"])[2]', driver)
    safe_click('(//img[@id="fig2"])', driver)
    safe_click('//*[@name="SMês_Diagnóstico"]/option[contains(.,"Todas as categorias")]', driver)
    safe_click(f'//*[@name="SMês_Diagnóstico"]/option[contains(.,"{mes}")]', driver)
    safe_click('(//img[@id="fig43"])', driver)
    time.sleep(1)
    safe_click('//*[@name="STipo_de_entrada__"]/option[contains(.,"CASO NOVO")]', driver)
    time.sleep(1)
    safe_click('//*[@name="STipo_de_entrada__"]/option[contains(.,"NÃO SABE")]', driver)
    time.sleep(1)
    safe_click('//*[@name="STipo_de_entrada__"]/option[contains(.,"PÓS ÓBITO")]', driver)
    time.sleep(1)
    safe_click('//*[@name="STipo_de_entrada__"]/option[contains(.,"Todas as categorias")]', driver)

    safe_click('//*[@value="prn"]', driver)
    safe_click('//input[@class="mostra"]', driver)

    dados = driver.find_element_by_xpath('//div[@name="governo"]')
    html = dados.get_attribute("innerHTML")
    soup = BeautifulSoup(html, "html.parser") 
    #mydivs = soup.findAll('text') 
    mydivs = soup.findAll("pre") 
    data = []
    for div in mydivs:
        data.append(div.text)


    lale = (data[0].split('\n'))

    lista_dados = []
    for i in list(range(0, 28)):
        lista_dados.append(lale[i].split(';'))

    print(f'Finished {mes}')
    df_mes = pd.DataFrame(lista_dados[1:], columns=lista_dados[0])
    pedrao['mes'] = mes
    ratosus = pd.concat([ratosus, df_mes])

WebDriverException: Message: 'chromedriver' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home


In [7]:
ratosus

,"""UF de notificação""","""2001""","""2002""","""2003""","""2004""","""2005""","""2006""","""2007""","""2008""","""2009""",...,"""2012""","""2013""","""2014""","""2015""","""2016""","""2017""","""2018""","""2019""","""Total""",mes
0,"""11 Rondônia""",56,52,48,29,33,27,35,45,37,...,56,45,56,43,50,47,42,42,817,Janeiro
1,"""12 Acre""",31,40,32,29,34,37,17,18,37,...,40,27,54,22,34,31,23,35,592,Janeiro
2,"""13 Amazonas""",208,184,190,166,162,153,179,179,184,...,173,247,229,229,219,247,255,272,3847,Janeiro
3,"""14 Roraima""",11,7,12,13,8,8,9,12,16,...,10,9,12,19,13,16,16,23,228,Janeiro
4,"""15 Pará""",269,270,280,312,287,261,296,288,288,...,279,323,312,281,296,301,300,345,5586,Janeiro
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22,"""43 Rio Grande do Sul""",344,421,431,426,372,346,373,455,388,...,380,372,381,380,415,354,370,272,7354,Dezembro
23,"""50 Mato Grosso do Sul""",52,54,81,76,70,59,64,70,53,...,71,54,53,48,66,65,83,34,1183,Dezembro
24,"""51 Mato Grosso""",87,74,88,76,86,70,79,73,66,...,116,104,79,132,71,71,95,28,1568,Dezembro
25,"""52 Goiás""",69,68,75,64,77,68,43,50,75,...,58,55,61,83,78,69,61,39,1234,Dezembro


In [ ]:
'''
actions = ActionChains(driver)
actions.key_down(keys.Keys.CONTROL)
a = driver.find_element_by_xpath('//*[@name="STipo_de_entrada__"]/option[contains(.,"NÃO SABE")]')
actions.click(a)
actions.key_up(keys.Keys.CONTROL)
actions.perform()

time.sleep(1)
actions = ActionChains(driver)
actions.key_down(keys.Keys.CONTROL)
a = driver.find_element_by_xpath('//*[@name="STipo_de_entrada__"]/option[contains(.,"PÓS ÓBITO")]')
actions.click(a)
actions.key_up(keys.Keys.CONTROL)
actions.perform()
'''